In [453]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.schema import AIMessage
import json
import re
from langchain_core.runnables import RunnableLambda
import base64
from PIL import Image
import io
import os
import pandas as pd
from tabulate import tabulate

In [454]:
llm = ChatOllama(model="gemma3")
directory = "photos_victor"

In [455]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [456]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [457]:
def prompt_func(data):
    type_ = data["type"]
    text = data["text"]
    content_parts = []

    if type_ == "keywords":
        image = data["image"]
        image_part = {
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{image}",
        }
        content_parts.append(image_part)

    #system_message = SystemMessage(content=data["system_message_text"])

    text_part = {"type": "text", "text": text}

    content_parts.append(text_part)

    human_message = HumanMessage(content=content_parts)

    #return [system_message, human_message]
    return [human_message]

In [458]:
def call_func(chain, prompt):
    try:
        response = chain.invoke(prompt)

        if isinstance(response, AIMessage):
            response_text = response.content
        else:
            response_text = str(response)  # Conversion en string si nécessaire

        #print(f"Reponse du llm : {response_text}")

        return extract_json(response_text)

    except Exception as e:
        print(f"Erreur de parsing JSON : {e}. Nouvelle tentative...")
        return -1

In [459]:
def get_image_paths(directory):
    allowed_extensions = {".jpg", ".jpeg", ".png"}
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.splitext(filename)[1].lower() in allowed_extensions]
    return image_paths

In [460]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])
    df["keywords"] = ""
    df["categories"] = ""

    return df

In [461]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        # Mise à jour uniquement pour les images présentes dans keywords_output
        image_data["keywords"] = image_data.apply(
            lambda row: keywords_output[row["image_name"]]
            if row["image_name"] in keywords_output else row["keywords"], axis=1
        )
    else:
        print("Aucun mot clé fourni ! ")
    return image_data

In [462]:
def add_categories_to_df(image_data, categories_output):
    if categories_output:
        #Inversion du dict : on associe une categorie a chaque image
        image_to_categories = {img: cat for cat, images in categories_output.items() for img in images}

        image_data.loc[image_data["image_name"].isin(image_to_categories.keys()), "categories"] = image_data["image_name"].map(image_to_categories)
    else:
        print("Aucune catégorisation trouvée !")

    return image_data

In [463]:
def get_missing_values(dictionnary):
    missing_values = {}
    for key, value in dictionnary.items():
        if value is None or value == "" or value == "nan" or value == "None":
            missing_values.update({key: value})

    return missing_values

In [464]:
def get_missing_path(paths, dictionnary):
    missing_paths = []
    for path in paths:
        image_name = os.path.split(path)[-1]
        #print(f"image_name : {image_name}")
        if image_name in dictionnary.keys() or path in dictionnary.keys():
            missing_paths.append(path)

    return missing_paths

In [465]:
def get_missing_values_path(paths, dictionnary):
    missing_values = get_missing_values(dictionnary)
    missing_paths = get_missing_path(paths, missing_values)
    print(f"Images détectées avec valeurs manquantes : {missing_values.keys()}")
    print(f"Chemins renvoyés pour traitement : {missing_paths}")
    return missing_paths

In [466]:
def checking_all_keywords(df):
    path_images_empty = []
    none_possibilities = [None, "", [], "None", ["None"]]
    for row in df.itertuples():
        keywords = row.keywords
        if isinstance(keywords, float) and pd.isna(keywords):
            path_images_empty.append(row.path)

        elif keywords in none_possibilities:
            path_images_empty.append(row.path)

    return path_images_empty

In [467]:
def keywords_call(df, image_paths, keywords_chain):
    for i in range(0, len(image_paths)):
        print(f"Image {i} : {image_paths[i]}")
        image_b64 = encode_image(image_paths[i])
        image_name = os.path.basename(image_paths[i])
        #print("Image name donnée au model : ", image_name)

        wrong_json = True
        max_iter = 100
        while wrong_json and max_iter > 0:
            prompt = {
                "type": "keywords",
                "text": f"""Décris-moi l'image avec 5 mots-clés. Les mots-clés doivent en priorité inclure des actions, des objets et un lieu si identifiables. Les mots-clés doivent être en français et peuvent être des mots composés.
                Retourne le résultat au format JSON suivant: {{ "{image_name}" : ["mot-clé1", "mot-clé2", "mot-clé3", "mot-clé4", "mot-clé5"] }}""",
                "image": image_b64
            }

            keywords_output = call_func(keywords_chain, prompt)
            print(f"Keywords : {keywords_output}\n")

            if keywords_output is None:
                max_iter -= 1
                print(f"On re-essaie avec au maximum : {max_iter}\n")
            else:
                df = add_keywords_to_df(df, keywords_output)
                wrong_json = False

    return df

In [468]:
def pipeline_keywords(image_paths):
    image_data = create_df(image_paths)
    new_image_paths = image_paths

    prompt_chain = RunnableLambda(prompt_func)
    keyword_chain = prompt_chain | llm

    all_keywords = False
    only_once = False

    while not all_keywords :
        print("Entree dans le while")
        image_data = keywords_call(image_data, new_image_paths, keyword_chain)

        if only_once:
            new_row = pd.DataFrame([{"image_name" : "IMG_20241228_132157.jpg","path": "photos_victor/IMG_20241228_132157.jpg"}])
            image_data = pd.concat([image_data, new_row], ignore_index=True)
            only_once = False

        new_image_paths = checking_all_keywords(image_data)
        print(f"Images à traiter après le passage : {new_image_paths}")

        if not new_image_paths:
            all_keywords = True

    return image_data

In [469]:
image_paths = get_image_paths(directory)
image_data = pipeline_keywords(image_paths)

Entree dans le while
Image 0 : photos_victor\IMG_20241228_124125_1.jpg
Keywords : {'IMG_20241228_124125_1.jpg': ['Pluie', 'Rue', 'Vélo', 'Bâtiments', 'Ville']}

Image 1 : photos_victor\IMG_20241228_124128.jpg
Keywords : {'IMG_20241228_124128.jpg': ['Pluie', 'Bitume', 'Bâtiments', 'Passerelle', 'Ville']}

Image 2 : photos_victor\IMG_20241228_124135.jpg
Keywords : {'IMG_20241228_124135.jpg': ['vélo', 'circulation', 'immeuble', 'pluie', 'ville']}

Image 3 : photos_victor\IMG_20241228_124137.jpg
Keywords : {'IMG_20241228_124137.jpg': ['Cycliste', 'Rue', 'Bâtiments', 'Pluie', 'Ville']}

Image 4 : photos_victor\IMG_20241228_124140.jpg
Keywords : {'IMG_20241228_124140.jpg': ['Bitume', 'Vélo', 'Étoiles', 'Bâtiments', 'Ville']}

Image 5 : photos_victor\IMG_20241228_124648.jpg
Keywords : {'IMG_20241228_124648.jpg': ['Vitrine', 'Bâtiment', 'Blanc', 'Boutique', 'Éléments']}

Image 6 : photos_victor\IMG_20241228_132157.jpg
Keywords : {'IMG_20241228_132157.jpg': ['glace', 'chocolat', 'tartelette', '

In [470]:
print(tabulate(image_data, headers="keys", tablefmt="psql"))

+----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+--------------+
|    | image_name                     | path                                         | date_time           | localisation                                                                                                                                                         | keywords                                                                                                              | categories   |
|----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------

In [471]:
def checking_all_categories(df):
    keywords_empty_categories = {}
    none_possibilities = [None, "", [], "None", ["None"]]
    for row in df.itertuples():
        check = row.categories
        if isinstance(check, float) and pd.isna(check):
            keywords_empty_categories.update({row.image_name: row.keywords})
        elif check in none_possibilities:
            keywords_empty_categories.update({row.image_name: row.keywords})

    return keywords_empty_categories

In [472]:
def get_cat_list(df, initial_cat_list):
    cat_list = initial_cat_list
    for row in df.itertuples():
        if row.categories not in cat_list:
            cat_list.append(row.categories)

    return cat_list

In [473]:
def categories_call(df, keywords, limit_size, cat_list, cat_chain):
    for i in range(0, len(keywords), limit_size):
        interval = [i, min(i + limit_size, len(df))]

        subset_keys = list(keywords.keys())[interval[0]:interval[1]]

        subset_keywords = {key: keywords[key] for key in subset_keys}
        print(f"Keywords : {subset_keywords}")

        wrong_json = True
        max_iter = 100
        while wrong_json and max_iter > 0:
            prompt = {"type":"categories", "text":f"""
                1. Regroupe les images en fonction de l'action, évènement ou de l'activité qu'elles représentent.
                2. Chaque catégorie est définie par un seul mot-clé descriptif.
                3. Une image ne peut appartenir qu'à une seule catégorie.
                4. Priorise les catégories existantes : {cat_list}. Si une image correspond à l'une d'elles, classe-la dedans.
                5. Si aucune catégorie existante ne convient, crée une nouvelle catégorie proche d’une activité de voyage ou une catégorie plus générique (ex: "Nature", "Repas", "Loisirs")
                6. Réduis autant que possible la catégorie "Autres". N’y mets une image que si elle est vraiment impossible à classer ailleurs.

                - Listes de mots-clés détectés pour chaque image dans le format dict = "image_name": [mots-cles] : {subset_keywords}

                Retourne uniquement le résultat et au format JSON : {{ "categorie1": [ "image_name", "image_name" ], "categorie2": [ "image_name", "image_name" ],...}}"""
            }

            categories_output = call_func(cat_chain, prompt)
            print(f"Categories : {categories_output}\n")

            if categories_output is None:
                max_iter -= 1
                print(f"On re-essaie avec au maximum : {max_iter}\n")
            else:
                df = add_categories_to_df(df, categories_output)
                cat_list = get_cat_list(df, cat_list)
                print(f"Nouvelle liste des categories : {cat_list}")
                wrong_json = False

    return df

In [474]:
def pipeline_categories(image_data, limit_size=20):
    new_keywords = image_data.set_index("image_name")["keywords"].to_dict()
    #print(new_keywords)
    cat_list = ["Paysage", "Ville", "Plage", "Randonnée", "Sport", "Musée", "Restaurant", "Autres"]

    prompt_chain = RunnableLambda(prompt_func)
    cat_chain = prompt_chain | llm

    all_categories = False
    only_once = False

    while not all_categories:
        image_data = categories_call(image_data, new_keywords, limit_size, cat_list, cat_chain)

        if only_once:
            image_data.loc[1, "categories"] = None
            only_once = False


        new_keywords = checking_all_categories(image_data)
        print(f"Kewords à repasser après checking : {new_keywords}")
        cat_list = get_cat_list(image_data, cat_list)

        if not new_keywords:
            all_categories = True

    return image_data

In [475]:
copy_image_data = pipeline_categories(image_data, limit_size=22)

Keywords : {'IMG_20241228_124125_1.jpg': ['Pluie', 'Rue', 'Vélo', 'Bâtiments', 'Ville'], 'IMG_20241228_124128.jpg': ['Pluie', 'Bitume', 'Bâtiments', 'Passerelle', 'Ville'], 'IMG_20241228_124135.jpg': ['vélo', 'circulation', 'immeuble', 'pluie', 'ville'], 'IMG_20241228_124137.jpg': ['Cycliste', 'Rue', 'Bâtiments', 'Pluie', 'Ville'], 'IMG_20241228_124140.jpg': ['Bitume', 'Vélo', 'Étoiles', 'Bâtiments', 'Ville'], 'IMG_20241228_124648.jpg': ['Vitrine', 'Bâtiment', 'Blanc', 'Boutique', 'Éléments'], 'IMG_20241228_132157.jpg': ['glace', 'chocolat', 'tartelette', 'dessert', 'assiette'], 'IMG_20241228_141921.jpg': ['Tourisme', 'New York', 'Deux personnes', 'Passants', 'Signalisation'], 'IMG_20241229_123811.jpg': ['Avion', 'Exposition', 'Salle', 'Blanc', 'Échelle'], 'IMG_20241229_123828.jpg': ['Avion', 'Aile', 'Échafaudage', 'Maintenance', 'Structure'], 'IMG_20241229_123844.jpg': ['Avion', 'Naval', 'Exposition', 'Sol', 'Retour'], 'IMG_20241229_124813.jpg': ['Canon', 'Armement', 'Militaire', 'Ins

In [476]:
print(tabulate(copy_image_data, headers="keys", tablefmt="psql"))

+----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+--------------+
|    | image_name                     | path                                         | date_time           | localisation                                                                                                                                                         | keywords                                                                                                              | categories   |
|----+--------------------------------+----------------------------------------------+---------------------+----------------------------------------------------------------------

In [480]:
csv_name = directory + ".csv"
copy_image_data.to_csv(csv_name, index=False)